# Вебинар 6. Двухуровневые модели рекомендаций


Код для src, utils, metrics вы можете скачать из [этого](https://github.com/geangohn/recsys-tutorial) github репозитория

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from xgboost import XGBClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('../webinar_3/raw_data/retail_train.csv')
item_features = pd.read_csv('../webinar_3/raw_data/product.csv')
user_features = pd.read_csv('../webinar_3/raw_data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [5]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [176]:
recommender.get_als_recommendations(2500, N=5)

[1012913, 977854, 1121213, 1075637, 917381]

In [8]:
recommender.get_own_recommendations(2375, N=5)

[1036501, 1085983, 1079023, 907099, 910439]

In [9]:
recommender.get_similar_items_recommendation(2375, N=5)

[890689, 1046545, 9527160, 845208, 12385373]

In [175]:
recommender.get_similar_users_recommendation(2500, N=5)

[1077600, 6979753, 964594, 970866, 1064213]

### Задание 1

A) Попробуйте различные варианты генерации кандидатов. Какие из них дают наибольший recall@k ?
- Пока пробуем отобрать 50 кандидатов (k=50)
- Качество измеряем на data_val_lvl_1: следующие 6 недель после трейна

Дают ли own recommendtions + top-popular лучший recall?  

B)* Как зависит recall@k от k? Постройте для одной схемы генерации кандидатов эту зависимость для k = {20, 50, 100, 200, 500}  
C)* Исходя из прошлого вопроса, как вы думаете, какое значение k является наиболее разумным?


In [219]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [233]:
def do_recommend(model, func, n, k):
    
    func_dic = {
        'get_als_recommendations': model.get_als_recommendations,
        'get_own_recommendations': model.get_own_recommendations,
        'get_similar_items_recommendation': model.get_similar_items_recommendation,
        'get_similar_users_recommendation': model.get_similar_users_recommendation
    }
    
    try:
        result = func_dic[func](n, k)
    
    except (IndexError, ValueError) as e:
        
        result = recommender._extend_with_top_popular([], 50)
    
    return result

In [220]:
%%time 

k = 50
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_als_recommendations', x, k))

CPU times: user 58.9 s, sys: 1.78 s, total: 1min
Wall time: 32.3 s


In [224]:
%%time

result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_own_recommendations', x, k))

CPU times: user 19 s, sys: 1.45 s, total: 20.5 s
Wall time: 22.1 s


In [225]:
%%time

result_lvl_1['sim_item'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_similar_items_recommendation', x, k))

CPU times: user 27.6 s, sys: 921 ms, total: 28.5 s
Wall time: 17.5 s


In [231]:
%time

result_lvl_1['sim_user'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_similar_users_recommendation', x, k))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 8.11 µs
####################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [232]:
for col in result_lvl_1.columns[2:]:
    print(f"{col}: {result_lvl_1.apply(lambda row: recall_at_k(row[col], row['actual'], k=k), axis=1).mean()}")


als: 0.06054955716324515
own: 0.09296981152765454
sim_item: 0.04218629437980274
sim_user: 0.009779507203679596


In [234]:
# Ответ:
# наибольший recall  дает вариант get_own_recommendations

In [241]:
k = [20, 50, 100, 200, 500]

tmp_result = pd.DataFrame()

for i in k:
    tmp_result[i] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_own_recommendations', x, i))
    

In [ ]:
tmp_result['actual'] = result_lvl_1['actual']

In [259]:
cols = tmp_result.columns[:-1].to_list()
cols

[20, 50, 100, 200, 500]

In [262]:
for col in cols:
    print(f"{col}: {tmp_result.apply(lambda row: recall_at_k(row[col], row['actual'], k=col), axis=1).mean()}")


20: 0.056632214567550486
50: 0.09296981152765454
100: 0.13448355043842
200: 0.19404514529918684
500: 0.2690299098670662


### Задание 2.

Обучите модель 2-ого уровня, при этом:
    - Добавьте минимум по 2 фичи для юзера, товара и пары юзер-товар
    - Измерьте отдельно precision@5 модели 1-ого уровня и двухуровневой модели на data_val_lvl_2
    - Вырос ли precision@5 при использовании двухуровневой модели?

In [6]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2.head(2)

,user_id
0,2070
1,2021


In [7]:
# Пока только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
users_lvl_2

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."
2,1753,"[967041, 963686, 948640, 1057168, 942475, 9421..."
3,2120,"[5707857, 1029743, 1106523, 5569230, 916122, 8..."
4,1346,"[1135983, 5569309, 1129982, 5574377, 5569993, ..."
...,...,...
2149,1446,"[6391134, 12487356, 827332, 7024847, 13007435,..."
2150,1784,"[917406, 1131625, 849578, 950935, 886395, 9796..."
2151,436,"[953180, 918046, 915459, 934399, 1092363, 8787..."
2152,1697,"[993838, 922847, 886103, 7442505, 13133916, 10..."


In [8]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,1105426,1
0,2070,1097350,1
0,2070,879194,1
0,2070,948640,1


In [9]:
users_lvl_2.shape[0], users_lvl_2['user_id'].nunique()

(107550, 2151)

In [66]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [67]:
targets_lvl_2['target'].mean()

0.10460593102333061

#### GB

In [12]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [13]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [14]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [15]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [23]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [30]:
X_train = pd.get_dummies(X_train, columns=cat_feats, drop_first=True)

In [18]:
xgb_model = XGBClassifier(n_estimators=50, max_depth=3, random_state=42, verbosity=1)

In [31]:
%%time 

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [84]:
train_preds = xgb_model.predict_proba(X_train)

In [85]:
train_preds

array([[0.86495936, 0.13504066],
       [0.8612424 , 0.1387576 ],
       [0.87583065, 0.12416936],
       ...,
       [0.79541767, 0.20458235],
       [0.8149511 , 0.18504886],
       [0.87810534, 0.12189467]], dtype=float32)

### Финальный проект

Мы уже прошли всю необходимуб теорию для финального проекта. Проект осуществляется на данных из вебинара (данные считаны в начале ДЗ).
Рекомендуем вам **начать делать проект сразу после этого домашнего задания**
- Целевая метрика - precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте
- Вы сдаете код проекта в виде github репозитория и csv файл с рекомендациями 